# Code Optimization for Small Nonlinear Systems

__Recall:__

User-side optimizations are important because, for sufficiently difficult problems, most time will be spent inside your f function, the function you are trying to solve. “Efficient solvers" are those that reduce the required number of f calls to hit the error tolerance. The main ideas for optimizing your nonlinear solver code, or any Julia function, are the following:

- Make it non-allocating
- Use StaticArrays for small arrays
- Use broadcast fusion
- Make it type-stable
- Reduce redundant calculations
- Make use of BLAS calls
- Optimize algorithm choice

In [1]:
using NonlinearSolve

f(u, p) = u .* u .- p
u0 = [1.0, 1.0]
p = 2.0
prob = NonlinearProblem(f, u0, p)
sol = solve(prob, NewtonRaphson())

using BenchmarkTools

@benchmark solve(prob, NewtonRaphson())

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  19.400 μs … 233.500 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     22.800 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   25.211 μs ±   8.278 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▁▄█                                                          
  ▂████▇█▆▄▃▃▂▃▃▂▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  19.4 μs         Histogram: frequency by time         58.9 μs <

 Memory estimate: 9.17 KiB, allocs estimate: 191.

Step 1: make it non-allocating ("in-place" approach):

In [2]:
function f(du, u, p)
    du[1] = u[1] * u[1] - p
    du[2] = u[2] * u[2] - p
    return nothing
end

prob = NonlinearProblem(f, u0, p)
@benchmark sol = solve(prob, NewtonRaphson())

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  11.300 μs … 159.900 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     13.300 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.565 μs ±   4.788 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁▇█▂▃                                                       
  ▂▇██████▄▄▃▃▃▃▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  11.3 μs         Histogram: frequency by time         33.1 μs <

 Memory estimate: 5.52 KiB, allocs estimate: 108.

Notice how much faster this already runs! We can make this code even simpler by using the `.=` in-place broadcasting.

In [3]:
function f(du, u, p)
    du .= u .* u .- p
    return nothing
end

@benchmark sol = solve(prob, NewtonRaphson())

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  10.800 μs …  1.482 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     13.200 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.898 μs ± 16.409 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▅▇████▆▅▅▃▃▃▅▅▂▂▁▁▁▁ ▁▁▁▁                                  ▃
  █████████████████████████████▇█▇█▇▆▇▇▇▅▇▆▅▇▆▅▆▆▅▅▄▅▆▇▃▅▇▆▅▄ █
  10.8 μs      Histogram: log(frequency) by time      36.2 μs <

 Memory estimate: 5.52 KiB, allocs estimate: 108.